<a href="https://colab.research.google.com/github/manueh56/Pre-entrega-CoderHouse/blob/main/ProyectoDSParteI_Lahitte.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Presentación del proyecto**

**Abstracto con Motivación y Audiencia**

**Contexto Comercial y Analítico**

**Preguntas/Hipótesis a Resolver mediante el Análisis de Datos**

**Objetivo**

# **Lectura de datos**

**Librerias necesarias para el analisis de datos**

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

**Configuración del entorno de Python en Google Colab**

**Lectura del dataset**

In [6]:
df = pd.read_csv("https://raw.githubusercontent.com/manueh56/Pre-entrega-CoderHouse/refs/heads/main/Dataset%20CoderHouse.csv", delimiter=';')

In [7]:
display(df.head())

,Age,Gender,Weight (kg),Height (m),Max_BPM,Avg_BPM,Resting_BPM,Session_Duration (hours),Calories_Burned,Workout_Type,...,cal_from_macros,pct_carbs,protein_per_kg,pct_HRR,pct_maxHR,cal_balance,lean_mass_kg,expected_burn,Burns Calories (per 30 min)_bc,Burns_Calories_Bin
0,34.91,Male,65.27,1.62,188.58,157.65,69.05,1.00,1080.90,Strength,...,2139.59,0.500432,1.624.789.336.601.800,0.7412365096628462,0.8359847279669106,7.250.999.999.999.990,4.777.739.381.514.470,685.16,"7,26E+34",Medium
1,23.37,Female,56.41,1.55,179.43,131.75,73.18,1.37,1809.91,HIIT,...,1711.65,0.500850,15.140.932.458.783.900,0.5512470588235293,0.7342696316112133,-23.291.000.000.000.000,408.098.026.823.542,9.786.184.000.000.000,"1,02E+34",High
2,33.20,Female,58.98,1.67,175.04,123.95,54.96,0.91,802.26,Cardio,...,1965.92,0.500610,16.634.452.356.731.000,0.5745336442371753,0.708123857404022,805.74,4.463.557.970.100.270,6.545.266,"1,08E+36",High
3,38.69,Female,93.78,1.70,191.21,155.10,50.07,1.10,1450.79,HIIT,...,1627.28,0.499533,0.8620174877372574,0.7441547399744933,0.811150044453742,1206.21,6.300.743.237.900.560,773.63,"8,99E+34",High
4,45.09,Male,52.42,1.88,193.58,152.88,70.84,1.08,1166.40,Strength,...,26.592.300.000.000.000,0.500581,2.538.153.376.573.820,0.6684047580250936,0.7897510073354684,3.035.999.999.999.990,4.334.750.358.810.570,7.114.176.000.000.000,"5,26E+34",Low


**Análisis inicial del dataset**

**Descripción de variables**

# **Data Wrangling - Limpieza y transformación de datos**

**Valores duplicados**

**Valores nulos**

**Tratamiento de valores outliers**

**Otras transformaciones, normalización, fechas**

# **Análisis Exploratorio de datos**

**Análisis univariado**

**Análisis bivariado**

**Análisis multivariado**

**Conclusiones del EDA**